# Одна очередь - несколько процессов
Использование очереди задач, но теперь процессов у нас ограниченное количество, то есть каждый процесс будет брать задачу (число) из очереди, делать нужные вычисления и помещать число фибоначчи в общий список результатов.

In [ ]:
from multiprocessing import Process, Manager, Queue
from multiprocessing.managers import ListProxy
from fib_settings import MAX_FIB_NUMBER, ACTIVATE_PROCESS_COUNT
import time

def worker_process_func(tasks_queue: Queue, shared_list_object: ListProxy = None) -> None:
    """Функция, которая будет выполняться каждым отдельным процессом"""

    def fibbonaci(number: int):
        """Функция для нахождения числа фибоначчи"""

        if number == 0:
            return 0
        
        if number < 2:
            return 1
        
        fib_number: int = fibbonaci(number - 2) + fibbonaci(number - 1)
        return fib_number
    
    # Процесс будет работать до тех пор, пока очередь задач полностью не опустеет
    while not tasks_queue.empty():
        one_number_task: int = tasks_queue.get()

        fib_number = fibbonaci(one_number_task)

        # В общий объект списка делаем добавление нашего результата (числа фибоначчи)
        shared_list_object.append(fib_number)

def main():
    # Делаем создание объекта очереди
    tasks_queue: Queue = Queue()

    # Делаем добавление в очередь задач чисел для которых нужно будет найти числа фибоначчи
    for one_number_task in range(0, MAX_FIB_NUMBER+1):
        tasks_queue.put(one_number_task)

    # Создаем общий объект списка результатов для того, чтобы процессы после высчитывания чисел 
    # фибоначчи, могли положить их туда, а мы после в главном процессе получить список результатов
    manager_object = Manager()
    shared_results_list = manager_object.list()


    process_list = []

    started_time = time.time()

    # Делаем создание списка процессов, указывая количество процессов как константу
    for one_process_index in range(0, ACTIVATE_PROCESS_COUNT):
        # Указываем как аргументы очередь задач и общий объект списка для результатов
        process_object = Process(target=worker_process_func, args=(
            tasks_queue, shared_results_list
        ))
        process_list.append(process_object)

    # Делаем перебор нашего списка процессов, после делаем запуск каждого процесса
    for one_process in process_list:
        one_process.start()

    # Делаем перебор списка процессов, после делаем слияние каждого дочернего процесса с главным 
    # процессом, то есть делаем ожидание завершения работы всех процессов
    for one_process in process_list:
        one_process.join()

    end_time = time.time()

    results_list: list = list(shared_results_list)

    print(f"Время всех вычислений: {end_time - started_time} секунд")
    print(results_list)
    print("После сортировки: ")
    print(sorted(results_list))


# Если этот файл был запущен из консоли
if __name__ == "__main__":
    main()